In [1]:
import matplotlib.pyplot as plt
import time
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dropout, Embedding, SimpleRNN, LSTM, GRU, Bidirectional, Dense, RNN, Flatten
from keras.regularizers import l2

from kerastuner.tuners import RandomSearch
from keras.optimizers import Adam, SGD, RMSprop, Nadam, Adagrad

2024-07-28 10:54:26.868417: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/zb/nl25s0l50sl0r6n18b3fm3w00000gp/T/ipykernel_98179/3303139053.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# Data analysis and preparation

In [8]:
max_features = 10000  # кількість слів, що розглядаються як особливості
maxlen = 500  # обмеження кількості слів в тексті
batch_size = 32

(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)


# Получение словаря индексов и слов
word_index = imdb.get_word_index() 
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in input_train[10]])
print(decoded_review)


### Padding
input_train = pad_sequences(input_train, maxlen=maxlen)
input_test = pad_sequences(input_test, maxlen=maxlen)

? french horror cinema has seen something of a revival over the last couple of years with great films such as inside and ? romance ? on to the scene ? ? the revival just slightly but stands head and shoulders over most modern horror titles and is surely one of the best french horror films ever made ? was obviously shot on a low budget but this is made up for in far more ways than one by the originality of the film and this in turn is ? by the excellent writing and acting that ensure the film is a winner the plot focuses on two main ideas prison and black magic the central character is a man named ? sent to prison for fraud he is put in a cell with three others the quietly insane ? body building ? marcus and his retarded boyfriend daisy after a short while in the cell together they stumble upon a hiding place in the wall that contains an old ? after ? part of it they soon realise its magical powers and realise they may be able to use it to break through the prison walls br br black magi

# Building and Comparing Recurrent Models

In [5]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_names = []
test_accuracies = [] 
test_losses = []

# Установим коллбэки для остановки обучения при отсутствии улучшений и уменьшения скорости обучения
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)

def plot_acc_loss_history(history, title=""):
    plt.figure(figsize=(13,3))

    plt.subplot(1,2,1)
    plt.plot(history.history["accuracy"], label="Training Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")  
    plt.fill_between(range(len(history.history['accuracy'])), history.history['accuracy'], history.history['val_accuracy'], alpha=0.2)
    plt.grid()
    plt.xticks(range(1, len(history.history['accuracy'])+1))
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.fill_between(range(len(history.history['loss'])), history.history['loss'], history.history['val_loss'], alpha=0.2)
    plt.grid()
    plt.xticks(range(1, len(history.history['loss'])+1))
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    if len(title):
        plt.suptitle(title, fontsize=20)
    plt.show()


def make_predictions(model, name: str):

    predictions = model.predict(input_test)

    # Для отримання зрозумілішого вигляду прогнозів, їх можна перетворити в бінарний формат:
    binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

    # Для порівняння можна вивести деякі прогнозовані значення поряд з фактичними значеннями
    for i in range(10):
        print(f"Predicted: {binary_predictions[i]}, Actual: {y_test[i]}")

    # Додатковий крок: Оцінка моделі
    # Це дасть загальну точність моделі на тестових даних
    test_loss, test_acc = model.evaluate(input_test, y_test)
    print(name,' Test Loss: {}'.format(test_loss))
    print(name, ' Test Accuracy: {}'.format(test_acc))

### Selection of hyperparameters with Kerastuner

In [7]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(max_features, 
                        hp.Int('embedding_dim', min_value=16, max_value=64, step=16),
                        input_length=maxlen))
    
    model.add(GRU(hp.Int('gru_units', min_value=32, max_value=256, step=32), 
                  kernel_regularizer=l2(hp.Float('l2_reg', min_value=0.01, max_value=0.1, step=0.01)),
                  return_sequences=False))
    
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), 
                    activation='relu', 
                    kernel_regularizer=l2(hp.Float('l2_reg', min_value=0.01, max_value=0.1, step=0.01))))
    
    model.add(Dense(1, activation='sigmoid'))

    # Обираємо оптимізатор та швидкість навчання
    optimizer_choice = hp.Choice(
        "optimizer", ["adam", "rmsprop", "nadam"]
    )
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    if optimizer_choice == "adam":
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_choice == "rmsprop":
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_choice == "nadam":
        optimizer = Nadam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=["accuracy"],
    )

    return model


tuner = RandomSearch(
    hypermodel=build_model,  # Функція побудови моделі, яка є build_modelв нашому випадку.
    objective="val_accuracy",  # Назва цілі, яку потрібно оптимізувати (мінімізувати чи максимізувати)
    max_trials=7,  # Загальна кількість спроб, які потрібно виконати під час пошуку.
    executions_per_trial=5,  # Кількість моделей, які слід створити та підібрати для кожного випробування.
    # Якщо ви хочете отримати результати швидше, ви можете встановити executions_per_trial=1(один цикл навчання для кожної конфігурації моделі).
    overwrite=True,  # чи перезаписувати попередні результати в тому самому каталозі
    directory="my_dir",
    project_name="imdb",
)
tuner.search_space_summary()


# Поиск гиперпараметров с измерением времени
start_time = time.time()
tuner.search(
    input_train,
    y_train,
    epochs=25,  # Значение может быть изменено в зависимости от задачи и доступных ресурсов
    validation_data=(input_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)
end_time = time.time()
search_time = end_time - start_time
print(f"Hyperparameter search time: {search_time:.2f} seconds")

# Получение лучшей модели
best_model = tuner.get_best_models(num_models=1)[0]
print(best_model.summary())


# Model training and evaluation
b_start_time = time.time()
b_history = best_model.fit(
    input_train,
    y_train,
    epochs=20,  # Значение может быть изменено в зависимости от задачи и доступных ресурсов
    validation_data=(input_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)
b_end_time = time.time()
b_training_time = b_end_time - b_start_time

b_test_loss, b_test_acc = best_model.evaluate(input_test, y_test)
print(f"Model training time for MODEL with TUNER: {b_training_time:.2f} seconds")

plot_acc_loss_history(b_history, "GRU with Keras Tuner")
make_predictions(best_model, "GRU with Keras Tuner")

2024-07-28 10:55:18.448514: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-28 10:55:18.449872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-28 10:55:18.450664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Search space summary
Default search space size: 7
embedding_dim (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
gru_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
l2_reg (Float)
{'default': 0.01, 'conditions': [], 'min_value': 0.01, 'max_value': 0.1, 'step': 0.01, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'nadam'], 'ordered': False}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32            

2024-07-28 10:55:18.960367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-28 10:55:18.963844: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-28 10:55:18.966463: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/25


2024-07-28 10:55:19.569343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-28 10:55:19.570925: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-28 10:55:19.572339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-07-28 11:58:03.225480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-28 11:58:03.226946: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-28 11:58:03.228193: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

782/782 [==============================] - ETA: 0s - loss: 0.9984 - accuracy: 0.4962

2024-07-28 12:02:48.372434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-28 12:02:48.375868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-28 12:02:48.376936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

782/782 [==============================] - 379s 481ms/step - loss: 0.9984 - accuracy: 0.4962 - val_loss: 1.0607 - val_accuracy: 0.4994 - lr: 0.0086
Epoch 2/25
782/782 [==============================] - 355s 454ms/step - loss: 1.0596 - accuracy: 0.5007 - val_loss: 1.0574 - val_accuracy: 0.5000 - lr: 0.0086
Epoch 3/25
782/782 [==============================] - 351s 449ms/step - loss: 1.0583 - accuracy: 0.5002 - val_loss: 1.0581 - val_accuracy: 0.5000 - lr: 0.0086
Epoch 4/25
782/782 [==============================] - 350s 448ms/step - loss: 1.0581 - accuracy: 0.4960 - val_loss: 1.0578 - val_accuracy: 0.5000 - lr: 0.0086
Epoch 5/25
782/782 [==============================] - 353s 451ms/step - loss: 1.0580 - accuracy: 0.4992 - val_loss: 1.0578 - val_accuracy: 0.5000 - lr: 0.0086
Epoch 6/25
782/782 [==============================] - 348s 445ms/step - loss: 1.0497 - accuracy: 0.4976 - val_loss: 1.0496 - val_accuracy: 0.5000 - lr: 0.0017
Epoch 7/25
782/782 [==============================] - 349

KeyboardInterrupt: 